In [1]:
import numpy as np
import csv
import math
from math import sin, cos, sqrt, fabs, atan2

In [604]:
ROOT = '/home/rick/PythonWorkplace/CIGIT2021/data/622'
filename = '2020-11-01-gps.csv'
data = ROOT + '/' + filename

# wgs2gcj

In [470]:
def geohey_transformLon(x, y):
    ret = 300.0 + x + 2.0 * y + 0.1 * x * x +  0.1 * x * y + 0.1 * sqrt(fabs(x))
    ret = ret + (20.0 * sin(6.0 * x * np.pi) + 20.0 * sin(2.0 * x * np.pi)) * 2.0 / 3.0
    ret = ret + (20.0 * sin(x * np.pi) + 40.0 * sin(x / 3.0 * np.pi)) * 2.0 / 3.0
    ret = ret + (150.0 * sin(x / 12.0 * np.pi) + 300.0 * sin(x * np.pi / 30.0)) * 2.0 / 3.0
    return ret

In [471]:
def geohey_transformLat(x, y):
    ret = -100.0 + 2.0 * x + 3.0 * y + 0.2 * y * y + 0.1 * x * y + 0.2 * sqrt(fabs(x))
    ret = ret + (20.0 * sin(6.0 * x * np.pi) + 20.0 * sin(2.0 * x * np.pi)) * 2.0 / 3.0
    ret = ret + (20.0 * sin(y * np.pi) + 40.0 * sin(y / 3.0 * np.pi)) * 2.0 / 3.0
    ret = ret + (160.0 * sin(y / 12.0 * np.pi) + 320.0 * sin(y * np.pi / 30.0)) * 2.0 / 3.0
    return ret

In [472]:
# define ellipsoid
a = 6378245.0
f = 1 / 298.3
b = a * (1 - f)
ee = 1 - (b * b) / (a * a)

def wgs2gcj(wgs_longi, wgs_lati):
    d_lati = geohey_transformLat(wgs_longi - 105.0, wgs_lati - 35.0)
    d_longi = geohey_transformLon(wgs_longi - 105.0, wgs_lati - 35.0)
    rad_lati = wgs_lati / 180 * np.pi
    magic = sin(rad_lati)
    magic = 1 - ee * magic * magic
    sqrtMagic = sqrt(magic)
    d_lati = (d_lati * 180.0) / ((a * (1 - ee)) / (magic * sqrtMagic) * np.pi)
    d_longi = (d_longi * 180.0) / (a / sqrtMagic * cos(rad_lati) * np.pi)
    gcj_lati = wgs_lati + d_lati
    gcj_longi = wgs_longi + d_longi
    return (gcj_longi, gcj_lati)

# Read CSV

In [605]:
lati = []
longi = []
v = []
bus_list = ['渝BP2853','渝BP2880','渝BP2908','渝BP2917','渝BP3690','渝BW1272','渝BW1282','渝BW1857','渝BW1872','渝BW1875',\
           '渝BW1895','渝BW2209','渝BW2210','渝BW2233','渝BW2235','渝BW2237','渝BW2270','渝BW2289','渝BW2291','渝BW2297']
delete_list = []
dic = {'渝BP2853':[],'渝BP2880':[],'渝BP2908':[],'渝BP2917':[],'渝BP3690':[],'渝BW1272':[],'渝BW1282':[],'渝BW1857':[],'渝BW1872':[],'渝BW1875':[],\
       '渝BW1895':[],'渝BW2209':[],'渝BW2210':[],'渝BW2233':[],'渝BW2235':[],'渝BW2237':[],'渝BW2270':[],'渝BW2289':[],'渝BW2291':[],'渝BW2297':[]}  
keys = ['origin','first path','first direction','first time start','first time end',\
        'last path','last direction','last time start','last time end']

In [606]:
for i in bus_list:
    tmp = i.replace('渝','')
    globals()[tmp] = []
    tmp = i.replace('渝','tmp_')
    globals()[tmp] = []

In [607]:
with open(data, 'r') as f:     # encoding='gbk'
    reader = csv.reader(f)
    for row in reader:
        if row[0] in bus_list:
            key = row[0]
            tmp_row = row[2:7]
            del tmp_row[3]    # 有用的列：时间，纬度，经度，速度
            tmp_row[1] = float(tmp_row[1])  # 纬度
            tmp_row[2] = float(tmp_row[2])  # 经度
            tmp_row[3] = float(tmp_row[3])  # 速度
            
            gcj_coord = wgs2gcj(tmp_row[2], tmp_row[1])
            tmp_row[1] = gcj_coord[1]
            tmp_row[2] = gcj_coord[0]
            
            tmp_name_ls = row[0].replace('渝','')
            globals()[tmp_name_ls].append(tmp_row)

for i in bus_list:
    tmp_name_ls = i.replace('渝','')
    if len(globals()[tmp_name_ls]) == 0:
        del globals()[tmp_name_ls]
        delete_list.append(i)

for i in delete_list:
    bus_list.remove(i)

In [608]:
delete_list

[]

In [609]:
err = 0.0008   # 由于采样率太低，且有数据缺失的情况，err应设置的较大

# 车站坐标为gcj
lati_LJ = 29.758752
longi_LJ = 106.631117

lati_JN_dn = 29.63154      # up2dn
longi_JN_dn = 106.552032

lati_JN_up = 29.631969     # dn2up
longi_JN_up = 106.553034

lati_TB = 29.755115
longi_TB = 106.627873

print('LJ_lati:', [lati_LJ - err, lati_LJ + err])
print('LJ_longi:', [longi_LJ - err, longi_LJ + err])
print('JNdn_lati:', [lati_JN_dn - err, lati_JN_dn + err])
print('JNdn_longi:', [longi_JN_dn - err, longi_JN_dn + err])
print('JNup_lati:', [lati_JN_up - err, lati_JN_up + err])
print('JNup_longi:', [longi_JN_up - err, longi_JN_up + err])
print('TB_lati:', [lati_TB - err, lati_TB + err])
print('TB_longi:', [longi_TB - err, longi_TB + err])

LJ_lati: [29.757952, 29.759552000000003]
LJ_longi: [106.630317, 106.631917]
JNdn_lati: [29.63074, 29.632340000000003]
JNdn_longi: [106.551232, 106.552832]
JNup_lati: [29.631169, 29.632769000000003]
JNup_longi: [106.552234, 106.553834]
TB_lati: [29.754315, 29.755915]
TB_longi: [106.627073, 106.62867299999999]


In [610]:
for i in bus_list:
    tmp_name_dic = i.replace('渝','dic_')
    tmp_name_ls = i.replace('渝','')
    globals()[tmp_name_dic] = dict.fromkeys(keys,"")
    globals()[tmp_name_dic][keys[0]] = globals()[tmp_name_ls]   # trans origin to 'origin'
    
    # first location
    if 29.6 < globals()[tmp_name_ls][5][1] < 29.7 and 106.5 < globals()[tmp_name_ls][5][2] < 106.6:  
        globals()[tmp_name_dic][keys[2]] = 2  # down ---> up
    elif 29.7 < globals()[tmp_name_ls][5][1] < 29.8 and 106.6 < globals()[tmp_name_ls][5][2] < 106.7:
        globals()[tmp_name_dic][keys[2]] = 1  # up ---> down
        
    # last location
    if 29.6 < globals()[tmp_name_ls][-5][1] < 29.7 and 106.5 < globals()[tmp_name_ls][-5][2] < 106.6:  
        globals()[tmp_name_dic][keys[6]] = 1  # up ---> down
    elif 29.7 < globals()[tmp_name_ls][-5][1] < 29.8 and 106.6 < globals()[tmp_name_ls][-5][2] < 106.7:
        globals()[tmp_name_dic][keys[6]] = 2  # down ---> up
    
    
    # first time & last time
    for row in globals()[tmp_name_ls]:
        
        # first direction
        if globals()[tmp_name_dic][keys[2]] == 2:  # down ---> up for first direction
            # 金南start
            if lati_JN_up - err < row[1] < lati_JN_up + err and longi_JN_up - err < row[2] < longi_JN_up + err:
                if globals()[tmp_name_dic][keys[3]] == '':
                    globals()[tmp_name_dic][keys[3]] = row[0]
                else:
                    continue
            # 桃北end           
            if lati_TB - err < row[1] < lati_TB + err and longi_TB - err < row[2] < longi_TB + err:
                if globals()[tmp_name_dic][keys[4]] == '':
                    globals()[tmp_name_dic][keys[4]] = row[0]
                else:
                    continue
            
        elif globals()[tmp_name_dic][keys[2]] == 1:  # up ---> down for first direction
            # 两江start
            if lati_LJ - err < row[1] < lati_LJ + err and longi_LJ - err < row[2] < longi_LJ + err:
                if globals()[tmp_name_dic][keys[3]] == '':
                    globals()[tmp_name_dic][keys[3]] = row[0]
                else:
                    continue
            # 金南end
            if lati_JN_dn - err < row[1] < lati_JN_dn + err and longi_JN_dn - err < row[2] < longi_JN_dn + err:
                if globals()[tmp_name_dic][keys[4]] == '':
                    globals()[tmp_name_dic][keys[4]] = row[0]
                else:
                    continue
            
    for row in globals()[tmp_name_ls]:
        
        # last direction
        if globals()[tmp_name_dic][keys[6]] == 2:  # down ---> up for last direction
            # 金南start
            if lati_JN_up - err < row[1] < lati_JN_up + err and longi_JN_up - err < row[2] < longi_JN_up + err:
                globals()[tmp_name_dic][keys[7]] = row[0]
            # 桃北end
            if lati_TB - err < row[1] < lati_TB + err and longi_TB - err < row[2] < longi_TB + err:
                globals()[tmp_name_dic][keys[8]] = row[0]
            
        elif globals()[tmp_name_dic][keys[6]] == 1:  # up ---> down for last direction
            # 两江start
            if lati_LJ - err < row[1] < lati_LJ + err and longi_LJ - err < row[2] < longi_LJ + err:
                globals()[tmp_name_dic][keys[7]] = row[0]
            # 金南end
            if lati_JN_dn - err < row[1] < lati_JN_dn + err and longi_JN_dn - err < row[2] < longi_JN_dn + err:
                globals()[tmp_name_dic][keys[8]] = row[0]
        


In [611]:
dic_BP2880

{'origin': [['2020-11-01 05:54:59',
   29.612164398481436,
   106.55260564912444,
   0.0],
  ['2020-11-01 05:55:09', 29.612164398481436, 106.55260564912444, 0.0],
  ['2020-11-01 05:54:29', 0.0027284583222547923, 0.015498497079753525, 0.0],
  ['2020-11-01 05:54:39', 0.0027284583222547923, 0.015498497079753525, 0.0],
  ['2020-11-01 05:54:49', 29.612164398481436, 106.55260564912444, 0.0],
  ['2020-11-01 05:55:19', 29.612164398481436, 106.55260564912444, 0.0],
  ['2020-11-01 05:55:40', 29.612164398481436, 106.55260564912444, 0.0],
  ['2020-11-01 05:55:50', 29.612164398481436, 106.55260564912444, 0.0],
  ['2020-11-01 05:56:10', 29.612164398481436, 106.55260564912444, 0.0],
  ['2020-11-01 05:56:20', 29.612164398481436, 106.55260564912444, 0.0],
  ['2020-11-01 05:56:30', 29.612164398481436, 106.55260564912444, 0.0],
  ['2020-11-01 05:56:41', 29.612164398481436, 106.55260564912444, 0.0],
  ['2020-11-01 05:56:51', 29.612164398481436, 106.55260564912444, 0.0],
  ['2020-11-01 05:57:01', 29.612164

In [612]:
for i in bus_list:
    tmp_name_dic = i.replace('渝','dic_')
    tmp_name_ls = i.replace('渝','')
    globals()[tmp_name_dic][keys[1]] = []
    globals()[tmp_name_dic][keys[5]] = []
    current1 = False
    current2 = False
    # for first path
    for row in globals()[tmp_name_ls]:
        while True:
            if row[0] != globals()[tmp_name_dic][keys[3]] and current1 == False:
                break
            elif row[0] == globals()[tmp_name_dic][keys[3]]:
                globals()[tmp_name_dic][keys[1]].append(row)
                current1 = True
                break
            elif row[0] != globals()[tmp_name_dic][keys[3]] and current1 == True:
                globals()[tmp_name_dic][keys[1]].append(row)
                if row[0] == globals()[tmp_name_dic][keys[4]]:
                    current1 = False
                break

                
    
    # for last path
    for row in globals()[tmp_name_ls]:
        while True:
            if row[0] != globals()[tmp_name_dic][keys[7]] and current2 == False:
                break
            elif row[0] == globals()[tmp_name_dic][keys[7]]:
                globals()[tmp_name_dic][keys[5]].append(row)
                current2 = True
                break
            elif row[0] != globals()[tmp_name_dic][keys[7]] and current2 == True:
                globals()[tmp_name_dic][keys[5]].append(row)
                if row[0] == globals()[tmp_name_dic][keys[8]]:
                    current2 = False
                break


# Save CSV

In [613]:
date = filename.replace(filename[10:], "", 1)
date

'2020-11-01'

In [614]:
for i in bus_list:
    name  = i.replace('渝','')
    
    tmp_name_dic = i.replace('渝','dic_')
    
    frt_direc = globals()[tmp_name_dic][keys[2]]
    lst_direc = globals()[tmp_name_dic][keys[6]]
    addr = '/home/rick/PythonWorkplace/CIGIT2021/data/622/split_2/'
    
    if isinstance(frt_direc, int) == True:
        csv_name_1 = date + '_' + name + '_' + '1' + '_' + str(frt_direc) + '.csv'   # first
        with open(addr + csv_name_1, 'w') as f:
            f_csv = csv.writer(f)
            f_csv.writerows(globals()[tmp_name_dic][keys[1]])
    else:
        continue
    
    if isinstance(lst_direc, int) == True:
        csv_name_2 = date + '_' + name + '_' + '2' + '_' + str(lst_direc) + '.csv'   # last
        with open(addr + csv_name_2, 'w') as f:
            f_csv = csv.writer(f)
            f_csv.writerows(globals()[tmp_name_dic][keys[5]])
    else:
        continue

# Definition of 'number or not' func

In [468]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
 
    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
 
    return False

# Definition of 'Computation of distance from longitude and latitude' func

In [4]:
from math import sin, asin, cos, radians, fabs, sqrt
 
EARTH_RADIUS=6371           
 
def hav(theta):
    s = sin(theta / 2)
    return s * s
 
def get_distance_hav(lat0, lng0, lat1, lng1):
    lat0 = radians(lat0)
    lat1 = radians(lat1)
    lng0 = radians(lng0)
    lng1 = radians(lng1)
 
    dlng = fabs(lng0 - lng1)
    dlat = fabs(lat0 - lat1)
    h = hav(dlat) + cos(lat0) * cos(lat1) * hav(dlng)
    distance = 2 * EARTH_RADIUS * asin(sqrt(h))
 
    return distance